In [ ]:
library(ComplexHeatmap)

In [ ]:
library(stringr)
library(dplyr)

In [4]:
library(readxl)

In [7]:
load('../02.differential/Deseq2_TvsN_FC2.Rdata')

## check immune checkpoints

In [8]:
Sti_immunecheckpoint <- c("CD27","CD70",## stimulatory, T cell receptor, APC ligand
                      "CD28","CD80","CD86",## stimulatory, CD4+ T cell receptor, APC ligand *2       
                      "CD40","CD40LG",## stimulatory,  CD4+ T cell ligand,APC cell receptor
                      "IL2RB","IL2",## stimulatory, receptor, CD4+ Tcell ligand
                      "TNFRSF9","CDC37L1",## stimulatory, Tcell receptor, APC ligand
                      "TNFRSF4","TNFSF4",## stimulatory, (OX40:OX40L) Tregs Receptor, APC ligand
                      "TNFRSF18","TNFSF18",## stimulatory (aka,GITR), CD8+ T cell Receptor, APC ligand 
                      "ICOS","ICOSLG",## stimulatory, CD4+ T cell Receptor, APC ligand
                       "CD276", ## No receptory, APC ligand
                        "TMIGD2","HHLA2"### Receptor , APC
                      )

In [9]:
Inh_immunecheckpoint <- c("CTLA4","CD80","CD86",## Inhibitory,Tregs receptor, APC ligand *2
                      "ADORA2A","ADORA2B",##Inhibitory , Tcell receptor, ligand: adenosine
                      "BTLA","TNFRSF14", ##Inhibitory, aka TNFRSF14 HVEM, Tcell receptor, APC ligand
                      "IDO1", ###Inhibitory, NA ,APC enzyme
                      "LAG3","FGL1", ###Inhibitory, Tcell receptor,APC ligand.
                      "CYBB", ##Inhibitory enzyme
                      "PDCD1","CD274","PDCD1LG2",##Inhibitory Tcell receptor, APC ligand
                      "HAVCR2","HMGB1","LGALS9","CEACAM1",##aka,TIM-3,Inhibitory, Tcell receptor, APC ligand *3
                      "VSIR","IGSF11",##Inhibitory, Tcell receptor, APC ligand
                      "SIGLEC7",##Inhibitory, Tcell receptor , ligand: sugar
                      "TIGIT","PVR","NECTIN2",##Inhibitory, Tcell ligand, APC recetpor
                      "SIRPA","CD47" ##Inhibitory, immune ligand, APC recetpor
                      )

In [10]:
rlt_high_tn$all[c(Sti_immunecheckpoint,Inh_immunecheckpoint),] -> rlt_high_tn_ic
rlt_low_tn$all[c(Sti_immunecheckpoint,Inh_immunecheckpoint),] -> rlt_low_tn_ic
rlt_neg_tn$all[c(Sti_immunecheckpoint,Inh_immunecheckpoint),] -> rlt_neg_tn_ic

In [11]:
openxlsx::write.xlsx(list('High' = rlt_high_tn_ic,
                          'Low' = rlt_low_tn_ic,
                          'Neg' = rlt_neg_tn_ic),
           rowNames = TRUE, file = "13.immune_checkpoint/ImmuneCheckpoint.xlsx")

Warning message in file.create(to[okay]):
“cannot create file '13.immune_checkpoint/ImmuneCheckpoint.xlsx', reason 'No such file or directory'”


In [12]:
get_data <- function(data){
    
    sti_2 <- data$all[Sti_immunecheckpoint[c(2,4,5,7,9,11,13,15,17,18,20)],][,c('log2FoldChange','padj')]## apc
    sti_1 <- data$all[Sti_immunecheckpoint[c(1,3,3,6,8,10,12,14,16,100,19)],][,c('log2FoldChange','padj')]## receptor
    sti_im_ck <- cbind(sti_1,sti_2)
    colnames(sti_im_ck) <- c('LC1','Padj1','LC2','Padj2')
    rownames(sti_im_ck) <- paste0(rownames(sti_1),"_",rownames(sti_2))
    sti_im_ck_lc <- sti_im_ck[,c(1,3)]
    sti_im_ck_p <- sti_im_ck[,c(2,4)]
  #  sti_im_ck_p <- -log10(sti_im_ck_p)

    Inb_2 <- data$all[Inh_immunecheckpoint[c(2,3,100,100,7,8,10,100,13,14,16,17,18,20,100,23,24,26)],][,c('log2FoldChange','padj')]
    Inb_1 <- data$all[Inh_immunecheckpoint[c(1,1,4,  5,  6,100,9,11,12,12,15,15,15,19,21, 22,22,25)],][,c('log2FoldChange','padj')]
    Inb_im_ck <- cbind(Inb_1,Inb_2)
    colnames(Inb_im_ck) <- c('LC1','Padj1','LC2','Padj2')
    rownames(Inb_im_ck) <- paste0(rownames(Inb_1),"_",rownames(Inb_2))
    Inb_im_ck_lc <- Inb_im_ck[,c(1,3)]
    Inb_im_ck_p <- Inb_im_ck[,c(2,4)]
  #  Inb_im_ck_p <- -log10(Inb_im_ck_p)

    High_LC <- rbind(sti_im_ck_lc,Inb_im_ck_lc)
    High_p <- rbind(sti_im_ck_p,Inb_im_ck_p)
    
    return(list(LC=High_LC,P=High_p))
}


In [13]:
High_immc <- get_data(rlt_high_tn)
Low_immc <- get_data(rlt_low_tn)
Neg_immc <- get_data(rlt_neg_tn)

In [14]:
Immune_rec <- cbind(High_immc$LC$LC1,Low_immc$LC$LC1,Neg_immc$LC$LC1)
Apc_lig <- cbind(High_immc$LC$LC2,Low_immc$LC$LC2,Neg_immc$LC$LC2)
Immune_rec_p <- cbind(High_immc$P$Padj1,Low_immc$P$Padj1,Neg_immc$P$Padj1)
Apc_lig_p <- cbind(High_immc$P$Padj2,Low_immc$P$Padj2,Neg_immc$P$Padj2)

In [15]:
rownames(Immune_rec) <- High_immc$LC %>% rownames() %>% str_split("_") %>% lapply(.,function(x) x[1][1]) %>% unlist()
rownames(Apc_lig) <- High_immc$LC %>% rownames() %>% str_split("_") %>% lapply(.,function(x) x[2][1]) %>% unlist()
colnames(Immune_rec) <- c("High","Low","Neg")
colnames(Apc_lig) <- c("High","Low","Neg")

In [16]:
immc_name <- High_immc$LC %>% rownames() %>% str_split("_") %>% lapply(.,function(x) x[1][1]) %>% unlist()

In [17]:
immc_name %>% str_split("\\.") %>% lapply(.,function(x) x[1][1])%>% unlist() ->immc_name

In [18]:
apc_name <- High_immc$LC %>% rownames() %>% str_split("_") %>% lapply(.,function(x) x[2][1]) %>% unlist()

In [19]:
apc_name %>% str_split("\\.") %>% lapply(.,function(x) x[1][1])%>% unlist() ->apc_name

In [20]:
options(repr.plot.width = 3, repr.plot.height = 8)

In [21]:
ha_r = rowAnnotation(foo = anno_text(apc_name, location = 0, just = "left"))
ha_l = rowAnnotation(foo = anno_text(immc_name, location = 1, just = "right"))

In [22]:
col_fun = circlize::colorRamp2(c(max(na.omit(Apc_lig)), 0, min(na.omit(Apc_lig))), c("red", "white", "green"))

In [23]:
lgd = Legend(labels = "FC>=1", type = "points",pch = 18, size = unit(5, 'mm'),
           direction = 'horizontal'
   )

In [24]:
heat <- Heatmap(Immune_rec, name = "FoldChange", col = col_fun, rect_gp = gpar(type = "none"), 
    cell_fun = function(j, i, x, y, width, height, fill) {
    
        if(Immune_rec_p[i,j]<=0.05 && !is.na(Immune_rec[i,j])){
            grid.rect(x = x, y = y, width = width, height = height, 
            gp = gpar(fill =col_fun(Immune_rec[i, j]), col = "grey"))
        #      grid.circle(x = x, y = y, r = abs(Immune_rec[i, j])/2 * min(unit.c(width, height)), 
         #       gp = gpar(fill = col_fun(Immune_rec[i, j]), col = NA))
             if(abs(Immune_rec[i,j])>=1){
                  grid.points(x = x, y = y, pch = 18, size = unit(4, "mm"))}
        }else{
            grid.rect(x = x, y = y, width = width, height = height, 
            gp = gpar(col = "grey", fill = NA))
            
        }        
     
    }, left_annotation = ha_l,
        cluster_rows = FALSE, cluster_columns = FALSE,
    show_row_names = FALSE, show_column_names = FALSE)

In [30]:
pdf(file = "Immune_checkpoint_immune_genes.pdf",width = 3,height = 8)
draw(heat,heatmap_legend_list = lgd)
dev.off()

png 
  2

In [31]:
heat2 <- Heatmap(Apc_lig, name = "FoldChange", col = col_fun, rect_gp = gpar(type = "none"), 
    cell_fun = function(j, i, x, y, width, height, fill) {
       # print(i)
        if(Apc_lig_p[i,j]<=0.05 && !is.na(Apc_lig[i,j])){
            grid.rect(x = x, y = y, width = width, height = height, 
            gp = gpar(fill =col_fun(Apc_lig[i, j]), col = "grey"))
         #      grid.circle(x = x, y = y, r = abs(Apc_lig[i, j])/2 * min(unit.c(width, height)), 
         #       gp = gpar(fill = col_fun(Apc_lig[i, j]), col = NA))
            if(abs(Apc_lig[i,j])>=1){
                  grid.points(x = x, y = y, pch = 18, size = unit(4, "mm"))
            }
        }else{
            grid.rect(x = x, y = y, width = width, height = height, 
            gp = gpar(col = "grey", fill = NA))
            
        }        
     
    }, right_annotation = ha_r,
        cluster_rows = FALSE, cluster_columns = FALSE,
    show_row_names = FALSE, show_column_names = FALSE)

In [32]:
pdf(file = "Immunecheckpoint_apc_genes.pdf",width = 3,height = 8)
draw(heat2,heatmap_legend_list = lgd)
dev.off()

png 
  2